# Travelling Salesperson Problem solved using genetic algorithms

In [1]:
# Imports 
import numpy as np
import random

from datetime import datetime

In [2]:
# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3

In [7]:
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Berlin': [50, 25],
 'London': [97, 72],
 'Moscow': [64, 59],
 'Barcelona': [84, 95],
 'Rome': [38, 26],
 'Paris': [63, 65],
 'Vienna': [54, 19],
 'Munich': [17, 39],
 'Istanbul': [86, 86],
 'Kyiv': [54, 24],
 'Bucharest': [77, 67],
 'Minsk': [74, 71],
 'Warsaw': [13, 80],
 'Budapest': [59, 53],
 'Milan': [43, 83],
 'Prague': [29, 66],
 'Sofia': [8, 9],
 'Birmingham': [32, 27],
 'Brussels': [26, 45],
 'Amsterdam': [5, 15]}

## 1. Create the first population set
We randomly shuffle the cities N times where N=population_size

In [13]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Rome', 'Munich', 'Warsaw', ..., 'Prague', 'London', 'Milan'],
       ['Sofia', 'Berlin', 'Vienna', ..., 'Amsterdam', 'Milan', 'Prague'],
       ['Brussels', 'Warsaw', 'Milan', ..., 'Prague', 'Kyiv', 'Sofia'],
       ...,
       ['Kyiv', 'Sofia', 'Berlin', ..., 'Milan', 'Minsk', 'Birmingham'],
       ['Istanbul', 'London', 'Brussels', ..., 'Paris', 'Rome', 'Minsk'],
       ['Paris', 'Berlin', 'Brussels', ..., 'Vienna', 'Birmingham',
        'London']], dtype='<U10')

## 2. Evaluate solutions fitness
The solutions are defined so that the first element on the list is the first city to visit, then the second, etc. and the last city is linked to the first.
The fitness function needs to compute the distance between subsequent cities.

In [9]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [14]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([ 950.61098449,  901.71317872,  918.93525615,  919.46593545,
        865.68170113, 1012.74637645, 1023.48202198, 1064.18687269,
        844.4843828 ,  910.86024671, 1031.22299292,  961.59142423,
        838.61283033,  959.6875044 ,  926.0434349 ,  815.94140974,
        997.21243021,  951.93623442, 1008.13760297,  910.77513276,
        976.49956553,  875.12642342,  811.36848624,  878.42265669,
        954.01087247,  984.98808202,  857.29379189,  947.23089683,
        917.19850968,  925.62907378,  995.77633436,  933.68960332,
        866.66964661,  778.20517673, 1019.2199845 ,  803.27797665,
        870.22573517,  845.17759872, 1044.08645085,  958.17802224,
        787.23298018,  994.87173809,  939.78734891,  849.80431065,
        890.2712228 , 1011.10321628, 1012.11648608,  910.89637361,
        862.40875879,  961.26649938,  971.10365455,  927.20868335,
        969.34937033,  780.6341453 ,  984.7990049 ,  919.93533129,
        962.88093231, 1023.23497176,  891.76524153, 1038.49382

# 3. Progenitors selection
I will select a new set of progenitors using the Roulette Wheel Selection. Generates a list of progenitor pairs where N= len(population_set) but at each position there are two solutions to merge

In [20]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['Moscow', 'Munich', 'Paris', 'Barcelona', 'Berlin', 'Prague',
       'London', 'Sofia', 'Budapest', 'Minsk', 'Istanbul', 'Amsterdam',
       'Bucharest', 'Birmingham', 'Rome', 'Milan', 'Kyiv', 'Warsaw',
       'Vienna', 'Brussels'], dtype='<U10')

# 4. Mating
For each pair of  parents we'll generate an offspring pair. Since we cannot repeat cities what we'll do is copy a random chunk from one progenitor and fill the blanks with the other progenitor.

In [33]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set

[array(['Berlin', 'Warsaw', 'Minsk', 'Brussels', 'Rome', 'Milan',
        'Amsterdam', 'London', 'Moscow', 'Birmingham', 'Vienna', 'Prague',
        'Istanbul', 'Barcelona', 'Paris', 'Sofia', 'Kyiv', 'Bucharest',
        'Budapest', 'Munich'], dtype='<U10'),
 array(['Kyiv', 'Sofia', 'Berlin', 'Vienna', 'Moscow', 'Birmingham',
        'Warsaw', 'Rome', 'Prague', 'Bucharest', 'Munich', 'Budapest',
        'Milan', 'Paris', 'Amsterdam', 'Istanbul', 'Barcelona', 'London',
        'Brussels', 'Minsk'], dtype='<U10'),
 array(['Moscow', 'Munich', 'Paris', 'Barcelona', 'Berlin', 'Bucharest',
        'Vienna', 'Istanbul', 'Amsterdam', 'Warsaw', 'Rome', 'Prague',
        'Minsk', 'Budapest', 'Brussels', 'Birmingham', 'Milan', 'Kyiv',
        'Sofia', 'London'], dtype='<U10'),
 array(['Prague', 'Sofia', 'Munich', 'Brussels', 'Paris', 'Minsk',
        'Birmingham', 'Bucharest', 'Warsaw', 'Kyiv', 'Rome', 'Barcelona',
        'Berlin', 'London', 'Budapest', 'Amsterdam', 'Milan', 'Istanbul',
        

# 5. Mutation
Now for each element of the new population we add a random chance of swapping

In [35]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop

[array(['Berlin', 'Warsaw', 'Brussels', 'Amsterdam', 'Istanbul',
        'Barcelona', 'Rome', 'London', 'Kyiv', 'Milan', 'Vienna', 'Moscow',
        'Budapest', 'Bucharest', 'Paris', 'Birmingham', 'Prague', 'Sofia',
        'Minsk', 'Munich'], dtype='<U10'),
 array(['Kyiv', 'Sofia', 'Berlin', 'London', 'Moscow', 'Warsaw',
        'Birmingham', 'Brussels', 'Amsterdam', 'Budapest', 'Vienna',
        'Munich', 'Milan', 'Rome', 'Prague', 'Barcelona', 'Istanbul',
        'Bucharest', 'Minsk', 'Paris'], dtype='<U10'),
 array(['Bucharest', 'Istanbul', 'Paris', 'Berlin', 'Barcelona',
        'Budapest', 'Minsk', 'London', 'Amsterdam', 'Warsaw', 'Vienna',
        'Prague', 'Munich', 'Rome', 'Kyiv', 'Birmingham', 'Milan',
        'Brussels', 'Sofia', 'Moscow'], dtype='<U10'),
 array(['London', 'Warsaw', 'Amsterdam', 'Brussels', 'Paris', 'Kyiv',
        'Barcelona', 'Birmingham', 'Rome', 'Berlin', 'Sofia', 'Bucharest',
        'Budapest', 'Prague', 'Minsk', 'Munich', 'Istanbul', 'Milan',
        

# 6. Stopping
To select the stopping criteria we'll need to create a loop to stop first. Then I'll set it to loop at 1000 iterations.

In [40]:
best_solution = [-1,np.inf,np.array([])]
for i in range(100000):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[0]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 651.1984110852003 904.5031133361371 31/01/21 10:20
100 650.2778159969758 917.9652570005769 31/01/21 10:20
200 728.0982067551198 916.6100957175964 31/01/21 10:20
300 690.3675738593695 916.1807799118312 31/01/21 10:20
400 695.7319541227392 914.2433960417703 31/01/21 10:20
500 740.2543579417166 912.3713728026829 31/01/21 10:20
600 710.8823923935223 914.6293407196176 31/01/21 10:20
700 670.476731642127 916.944884394244 31/01/21 10:20
800 657.6118052924968 908.7325981770912 31/01/21 10:20
900 701.8662803507286 920.5789974294747 31/01/21 10:20
1000 687.6682987694342 923.1843568754049 31/01/21 10:20
1100 693.8436886796943 925.7274725127462 31/01/21 10:20
1200 607.862050216369 915.1298175057091 31/01/21 10:20
1300 712.6096978086018 917.3141674304816 31/01/21 10:21
1400 704.8739311944677 906.9669315820591 31/01/21 10:21
1500 715.8246206532323 915.2358414834757 31/01/21 10:21
1600 713.7787848455639 923.2703905886863 31/01/21 10:21
1700 631.8800247922326 919.0932401164454 31/01/21 10:21
1800 64

14700 689.5663565242604 916.0384592261435 31/01/21 10:26
14800 700.027457837564 917.1824714212037 31/01/21 10:26
14900 703.2776251898622 916.043492358533 31/01/21 10:26
15000 656.1429991307231 919.0407652922358 31/01/21 10:26
15100 699.6706739037214 917.542636908571 31/01/21 10:27
15200 698.896753538404 919.6955165473533 31/01/21 10:27
15300 720.0636387093351 926.8785701581246 31/01/21 10:27
15400 695.5393288865802 919.0279503390188 31/01/21 10:27
15500 655.1086261871193 917.4305079773744 31/01/21 10:27
15600 655.664682927006 907.218011327906 31/01/21 10:27
15700 697.304090655483 920.1240298790366 31/01/21 10:27
15800 723.9492786491446 917.4382888086267 31/01/21 10:27
15900 711.2427582656478 928.0066199117202 31/01/21 10:27
16000 600.6561692685988 910.7692418913922 31/01/21 10:27
16100 712.6544046592007 918.1014383923243 31/01/21 10:27
16200 679.9664662952837 923.9372905480087 31/01/21 10:27
16300 654.3236927936046 923.577229424415 31/01/21 10:27
16400 668.041055689146 917.921333028353

29200 739.1253126810868 913.3052217159288 31/01/21 10:33
29300 679.5057085452622 897.2153196096334 31/01/21 10:33
29400 710.0324683512041 927.2994180277451 31/01/21 10:33
29500 752.4688963128995 924.8329332910724 31/01/21 10:33
29600 630.5000046092891 924.9461840379682 31/01/21 10:33
29700 689.8603295561618 927.2809915959168 31/01/21 10:33
29800 654.1025219319203 909.4528794435932 31/01/21 10:33
29900 705.7403487951289 921.7296074246525 31/01/21 10:33
30000 670.7719463381919 916.9444380436246 31/01/21 10:33
30100 735.4959928397486 909.0900829111408 31/01/21 10:33
30200 655.1265301082303 910.5325755656824 31/01/21 10:33
30300 718.4062873487851 929.4318543604106 31/01/21 10:33
30400 619.178881847699 909.4631269975227 31/01/21 10:33
30500 629.0864230097848 905.6145229150756 31/01/21 10:33
30600 707.2571686372727 931.9540620051143 31/01/21 10:33
30700 694.23696562032 931.9639046011607 31/01/21 10:33
30800 721.2061382707542 928.3655798999523 31/01/21 10:33
30900 645.7741097865041 904.165408

KeyboardInterrupt: 

In [41]:
best_solution

[491.16837179572036,
 array([['Amsterdam', 'Sofia', 'Munich', 'Birmingham', 'Warsaw', 'Prague',
         'Brussels', 'Kyiv', 'Vienna', 'Berlin', 'Rome', 'Paris',
         'Bucharest', 'Barcelona', 'Moscow', 'Milan', 'Budapest',
         'London', 'Istanbul', 'Minsk']], dtype='<U10')]